In [ ]:
import numpy as np
import math
import cv2
import imutils

def compute_skew(file_name):
    
    #load in grayscale:
    src = cv2.imread(file_name,0)
    height, width = src.shape[0:2]
    
    #invert the colors of our image:
    cv2.bitwise_not(src, src)
    
    #Hough transform:
    minLineLength = width/2.0
    maxLineGap = 20
    lines = cv2.HoughLinesP(src,1,np.pi/180,100,minLineLength,maxLineGap)
    
    #calculate the angle between each line and the horizontal line:
    angle = 0.0
    nb_lines = len(lines)
    
    
    for line in lines:
        angle += math.atan2(line[0][3]*1.0 - line[0][1]*1.0,line[0][2]*1.0 - line[0][0]*1.0);
    
    angle /= nb_lines*1.0
    
    return angle* 180.0 / np.pi


def deskew(file_name,angle):
    
    #load in grayscale:
    img = cv2.imread(file_name,0)
    
    #invert the colors of our image:
    cv2.bitwise_not(img, img)
    
    #compute the minimum bounding box:
    non_zero_pixels = cv2.findNonZero(img)
    center, wh, theta = cv2.minAreaRect(non_zero_pixels)
    
    root_mat = cv2.getRotationMatrix2D(center, angle, 1)
    rows, cols = img.shape
    rotated = cv2.warpAffine(img, root_mat, (cols, rows), flags=cv2.INTER_CUBIC)


    #Border removing:
    sizex = np.int0(wh[0])
    sizey = np.int0(wh[1])
    print(theta)
    if theta > -45 :
        temp = sizex
        sizex= sizey
        sizey= temp
    return cv2.getRectSubPix(rotated, (sizey,sizex), center)
  

file_path = 'generated_images/6.jpg'
#print(file_path.shape)
#rotated = imutils.rotate(cv2.imread(file_path), 90)
angel = compute_skew(file_path)
dst = deskew(file_path,angel)
cv2.imwrite('skew.jpg',dst)

#th2 = cv2.adaptiveThreshold(dst,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            #cv2.THRESH_BINARY,11,2)
#cv2.imwrite('skew.jpg',th2)


cv2.imshow("Result",dst)
cv2.waitKey(0)

In [ ]:
import numpy as np
import math
import cv2
import imutils

def compute_skew(file_name):
    
    #load in grayscale:
    src = cv2.imread(file_name,0)
    height, width = src.shape[0:2]
    
    #invert the colors of our image:
    cv2.bitwise_not(src, src)
    
    #Hough transform:
    minLineLength = width/2.0
    maxLineGap = 20
    lines = cv2.HoughLinesP(src,1,np.pi/180,100,minLineLength,maxLineGap)
    
    #calculate the angle between each line and the horizontal line:
    angle = 0.0
    nb_lines = len(lines)
    
    
    for line in lines:
        angle += math.atan2(line[0][3]*1.0 - line[0][1]*1.0,line[0][2]*1.0 - line[0][0]*1.0);
    
    angle /= nb_lines*1.0
    
    return angle* 180.0 / np.pi
  

def rotate_bound(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
 
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
 
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
 
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
 
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))


file_path = 'generated_images/1.jpg'
#print(file_path.shape)
#rotated = imutils.rotate(cv2.imread(file_path), 90)
angel = compute_skew(file_path)
print(angel)
dst = rotate_bound(cv2.imread(file_path),angel)
cv2.imwrite('skew.jpg',dst)

dst = cv2.fastNlMeansDenoisingColored(dst,None,10,10,7,21)

dst = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)

img_blur = cv2.medianBlur(dst,5).astype('uint8')
img_thresh_Gaussian = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11,2)

#cv2.imwrite('skew.jpg',th2)
cv2.imwrite('skew.jpg',img_thresh_Gaussian)

cv2.imshow("Result",img_thresh_Gaussian)
cv2.waitKey(0)


In [ ]:
#preprocessing step before performing ocr
import numpy as np
import math
import cv2
import imutils

file_path = 'generated_images/11.jpg'

dst = cv2.fastNlMeansDenoisingColored(cv2.imread(file_path),None,10,10,7,21)

dst = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)

img_blur = cv2.medianBlur(dst,5).astype('uint8')
img_thresh_Gaussian = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11,2)

cv2.imwrite('skew.jpg',img_thresh_Gaussian)
cv2.imshow('skew',img_thresh_Gaussian)
cv2.waitKey(0)

In [ ]:
#skew correction which works fine if the angle is not 45
# import the necessary packages
import numpy as np
import argparse
import cv2
import math
import imutils
import pytesseract
from PIL import Image

im = Image.open("hackathon_images/21.png")
im.save("skew.jpg", dpi=(300,300))

file_path = 'skew.jpg'
print(cv2.imread(file_path).shape)
#file_path = 'im_after_text_extraction.jpg'

#code to sharpen
#kernel_sharp = np.array([[-1,-1,-1],[-1, 9,-1],[-1,-1,-1]])
#dst = cv2.filter2D(cv2.imread(file_path),-1,kernel_sharp)

dst = cv2.fastNlMeansDenoisingColored(cv2.imread(file_path),None,10,10,7,21)

dst = cv2.medianBlur(dst,5).astype('uint8')
dst = cv2.GaussianBlur(dst,(5,5),cv2.BORDER_DEFAULT) 

dst = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
cv2.imwrite('pw.jpg',dst)

#cv2.imshow("k", dst)
#cv2.waitKey(0)
img_thresh_Gaussian = cv2.adaptiveThreshold(dst, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11,2)

#img_thresh_Gaussian = cv2.adaptiveThreshold(dst, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11,2)
ret2,img_thresh_Gaussian = cv2.threshold(img_thresh_Gaussian,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#cv2.imshow("k", img_thresh_Gaussian)
#cv2.waitKey(0)
#img_thresh_Gaussian = cv2.Canny(img_thresh_Gaussian,100,200)

cv2.imwrite('skew.jpg',img_thresh_Gaussian)

# load the image from disk
image = cv2.imread('skew.jpg')
	
# convert the image to grayscale and flip the foreground
# and background to ensure foreground is now "white" and
# the background is "black"
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

gray = cv2.bitwise_not(gray)


# threshold the image, setting all foreground pixels to
# 255 and all background pixels to 0
thresh = cv2.threshold(gray, 0, 255,
	cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

#cv2.imshow("Rotated", thresh)
#cv2.waitKey(0)
# grab the (x, y) coordinates of all pixel values that
# are greater than zero, then use these coordinates to
# compute a rotated bounding box that contains all
# coordinates
coords = np.column_stack(np.where(thresh > 0))
angle = cv2.minAreaRect(coords)[-1]
 
# the `cv2.minAreaRect` function returns values in the
# range [-90, 0); as the rectangle rotates clockwise the
# returned angle trends to 0 -- in this special case we
# need to add 90 degrees to the angle
if angle < -45:
	angle = -(90 + angle)
 
# otherwise, just take the inverse of the angle to make
# it positive
else:
	angle = -angle
print(angle)

if angle> -2 and angle < 5:
    angle = 0
# rotate the image to deskew it
(h, w) = image.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, angle, 1.0)
rotated = cv2.warpAffine(image, M, (w, h),
	flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

# draw the correction angle on the image so we can validate it
#cv2.putText(rotated, "Angle: {:.2f} degrees".format(angle),
#	(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
kernel_sharp = np.array([[-1,-1,-1],[-1, 9,-1],[-1,-1,-1]])
rotated = cv2.filter2D(rotated,-1,kernel_sharp)
rotated = cv2.medianBlur(rotated,5).astype('uint8')


if rotated[0][0][0]!=255 or rotated[0][0][1]!=255 or rotated[0][0][2]!=255:
    rotated = cv2.bitwise_not(rotated)

rotated = cv2.fastNlMeansDenoising(rotated, None, 9, 13)
#dst = cv2.filter2D(rotated,-1,kernel_sharp)
#rotated = cv2.fastNlMeansDenoising(rotated, None, 9, 13)
#rotated = cv2.resize(rotated,(int(rotated.shape[1]/2),int(rotated.shape[0]/2)))
cv2.imwrite('skew.jpg',rotated)
 
im1 = Image.open("skew.jpg")
print(pytesseract.image_to_string(im1, lang='eng'))
# show the output image
#print("[INFO] angle: {:.3f}".format(angle))
cv2.imshow("Input", image)
cv2.imshow("Rotated", rotated)
cv2.waitKey(0)

In [ ]:
import pytesseract
from PIL import Image
import cv2
import numpy as np
#kernel_sharp = np.array([[-1,-1,-1],[-1, 9,-1],[-1,-1,-1]])
#dst = cv2.filter2D(cv2.imread('hackathon_images/21.png'),-1,kernel_sharp)
#cv2.imwrite('hackathon_images/21.png',dst)
im1 = Image.open("hackathon_images/9.png")
print(pytesseract.image_to_string(im1, lang='eng'))

In [ ]:
# USAGE
# python text_detection.py --image images/lebron_james.jpg --east frozen_east_text_detection.pb

# import the necessary packages
import imutils
from imutils.object_detection import non_max_suppression
import numpy as np
#import argparse
import time
import cv2

# load the input image and grab the image dimensions
image = cv2.imread('fintech2.png')
orig = image.copy()
(H, W) = image.shape[:2]

# set the new width and height and then determine the ratio in change
# for both the width and height
(newW, newH) = (512,512)
rW = W / float(newW)
rH = H / float(newH)

# resize the image and grab the new image dimensions
image = cv2.resize(image, (newW, newH))
(H, W) = image.shape[:2]

# define the two output layer names for the EAST detector model that
# we are interested -- the first is the output probabilities and the
# second can be used to derive the bounding box coordinates of text
layerNames = [
	"feature_fusion/Conv_7/Sigmoid",
	"feature_fusion/concat_3"]

# load the pre-trained EAST text detector
print("[INFO] loading EAST text detector...")
net = cv2.dnn.readNet('frozen_east_text_detection.pb')

# construct a blob from the image and then perform a forward pass of
# the model to obtain the two output layer sets
blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
	(123.68, 116.78, 103.94), swapRB=True, crop=False)
start = time.time()
net.setInput(blob)
(scores, geometry) = net.forward(layerNames)
end = time.time()

# show timing information on text prediction
print("[INFO] text detection took {:.6f} seconds".format(end - start))

# grab the number of rows and columns from the scores volume, then
# initialize our set of bounding box rectangles and corresponding
# confidence scores
(numRows, numCols) = scores.shape[2:4]
rects = []
confidences = []

# loop over the number of rows
for y in range(0, numRows):
	# extract the scores (probabilities), followed by the geometrical
	# data used to derive potential bounding box coordinates that
	# surround text
	scoresData = scores[0, 0, y]
	xData0 = geometry[0, 0, y]
	xData1 = geometry[0, 1, y]
	xData2 = geometry[0, 2, y]
	xData3 = geometry[0, 3, y]
	anglesData = geometry[0, 4, y]

	# loop over the number of columns
	for x in range(0, numCols):
		# if our score does not have sufficient probability, ignore it
		if scoresData[x] < 0.7:
			continue

		# compute the offset factor as our resulting feature maps will
		# be 4x smaller than the input image
		(offsetX, offsetY) = (x * 4.0, y * 4.0)

		# extract the rotation angle for the prediction and then
		# compute the sin and cosine
		angle = anglesData[x]
		cos = np.cos(angle)
		sin = np.sin(angle)

		# use the geometry volume to derive the width and height of
		# the bounding box
		h = xData0[x] + xData2[x]
		w = xData1[x] + xData3[x]

		# compute both the starting and ending (x, y)-coordinates for
		# the text prediction bounding box
		endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
		endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
		startX = int(endX - w)
		startY = int(endY - h)

		# add the bounding box coordinates and probability score to
		# our respective lists
		rects.append((startX, startY, endX, endY))
		confidences.append(scoresData[x])

# apply non-maxima suppression to suppress weak, overlapping bounding
# boxes
boxes = non_max_suppression(np.array(rects), probs=confidences)
k=0
# loop over the bounding boxes
for (startX, startY, endX, endY) in boxes:
	# scale the bounding box coordinates based on the respective
	# ratios
	startX = int(startX * rW)
	startY = int(startY * rH)
	endX = int(endX * rW)
	endY = int(endY * rH)
	k = k+1 
	print(k)
	# draw the bounding box on the image
	im =  orig[startY:endY, startX:endX]
	cv2.imwrite('im_after_text_detection'+ str(k) +'.jpg',im)
	cv2.rectangle(orig, (startX-20, startY-20), (endX+20, endY+20), (0, 255, 0), 2)

# show the output image
cv2.imshow("Text Detection", orig)
cv2.waitKey(0)
cv2.destroyAllWindows(0)

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('generated_images/1.jpg')
img = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers+1

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv2.watershed(img,markers)
img[markers == -1] = [255,0,0]
cv2.imshow('segmented',img)
cv2.waitKey(0)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import color, data, restoration
from skimage.io import imsave

astro = color.rgb2gray(cv2.imread('pw.jpg'))
from scipy.signal import convolve2d as conv2
psf = np.ones((5, 5)) / 25
astro = conv2(astro, psf, 'same')
astro += 0.1 * astro.std() * np.random.standard_normal(astro.shape)

deconvolved, _ = restoration.unsupervised_wiener(astro, psf)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 5),
                       sharex=True, sharey=True)

plt.gray()

ax[0].imshow(astro, vmin=deconvolved.min(), vmax=deconvolved.max())
ax[0].axis('off')
ax[0].set_title('Data')

ax[1].imshow(deconvolved)
ax[1].axis('off')
ax[1].set_title('Self tuned restoration')

imsave('deconvoluted.jpg',deconvolved)
cv2.imshow('deconvolved', deconvolved)
cv2.waitKey(0)
fig.tight_layout()

plt.show()

In [ ]:
import cv2
import numpy as np
from PIL import Image
im = Image.open("fintech.jpg")
im.save("skew.jpg", dpi=(300,300))
img = cv2.imread('skew.jpg',0)
equ = cv2.equalizeHist(img)
res = np.hstack((img,equ)) #stacking images side-by-side
#equ = cv2.adaptiveThreshold(equ, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11,2)
kernel_sharp = np.array([[-1,-1,-1],[-1, 9,-1],[-1,-1,-1]])
equ = cv2.filter2D(equ,-1,kernel_sharp)
#thresh = cv2.threshold(equ, 0, 255,
#	cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
cv2.imwrite('sir1.jpg',equ)

#cv2.waitKey(0)

In [ ]:
#threshold_sauvola
import pytest
import numpy as np
from PIL import Image
from scipy import ndimage as ndi
import skimage
from skimage import data
from skimage import io
import cv2
import matplotlib.pyplot as plt
from skimage._shared._warnings import expected_warnings
from skimage.filters.thresholding import (threshold_local,
                                          threshold_otsu,
                                          threshold_li,
                                          threshold_yen,
                                          threshold_isodata,
                                          threshold_niblack,
                                          threshold_sauvola,
                                          threshold_mean,
                                          threshold_triangle,
                                          threshold_minimum,
                                          try_all_threshold,
                                          _mean_std)
from skimage._shared import testing
"""
def threshold_sauvola(image, window_size=15, k=0.2, r=None):
    if r is None:
        imin, imax = dtype_limits(image, clip_negative=False)
        r = 0.5 * (imax - imin)
    m, s = _mean_std(image, window_size)
    return m * (1 + k * ((s / r) - 1))
"""
def test_threshold_sauvola(image):
        thres = threshold_sauvola(image, window_size=3)
        out = image > thres
        return out
    
image = io.imread('generated_images/4.jpg',0)
thresh_sauvola1 = threshold_sauvola(image,3)
print(thresh_sauvola1)
binary_sauvola = image > thresh_sauvola1
binary_sauvola = 1 * binary_sauvola
print(binary_sauvola)

#img = Image.fromarray(binary_sauvola, 'RGB')
#cv2.imshow('g',binary_sauvola)
#cv2.waitKey(0)
#from scipy.misc import imshow
#imshow(binary_sauvola)

import scipy.misc
scipy.misc.imsave('outfilefff.jpg', binary_sauvola)

#plt.imshow(binary_sauvola, cmap=plt.cm.gray)
#plt.title('Sauvola Threshold')
#plt.axis('off')



In [1]:
from PIL import Image, ImageOps
 
 
def add_border(input_image, output_image, border):
    img = Image.open(input_image)
 
    if isinstance(border, int) or isinstance(border, tuple):
        bimg = ImageOps.expand(img, border=border)
    else:
        raise RuntimeError('Border is not an integer or tuple!')
 
    bimg.save(output_image)
 
if __name__ == '__main__':
    in_img = 'fin.png'
 
    add_border(in_img, output_image='butterfly_border.jpg',
               border=20)
    